## Purpose for Deployment

Deploying a social media website is essential for making it accessible to users beyond a developer’s local machine. Unlike running on localhost, deployment essential expands the user base web application meaning more people can access it. A deployed backend ensures the database can handle multiple users simultaneously, while cloud-based hosting enables scalability to support growing traffic. Additionally, deployment provides security features like HTTPS encryption and authentication, protecting user data from unauthorized access. 

## Deployment Steps

Prerequisites

- [x]   Review the backend-to-frontend functionality
- [X]   Document the deployment process with key steps and visuals
- [X]   Assign deployment admin roles
- [X]   Ensure configuration files (Dockerfile, docker-compose.yml, nginx config) are set 

Local Testing Before Deployment

- [X]   Verify the backend is fully functional locally
- [X]   Test CRUD operations using API calls
- [X]   Confirm the frontend correctly interacts with the backend

Domain and Subdomain Setup

- [X]   Register a subdomain using AWS Route 53
```yml
yml
Server: https://neptune_backend.nighthawkcodingsociety.com/
Domain: nighthawkcodingsociety.com
Subdomain: neptune_backend
```
- [X]   Configure the subdomain to route traffic correctly

Selecting Ports

- [X]   Choose an available backend port (we chose 8201 which doesn't conflict with classmate's ports)
- [X]   Update the port across all necessary files (example in main.py using new port number 8201)
```python
if __name__ == "__main__":
    app.run(debug=True, host="0.0.0.0", port="8201")
```
- [X]   Backend configuration
- [X]   Docker-related files
```Dockerfile
FROM docker.io/python:3.11
WORKDIR /
RUN apt-get update && apt-get upgrade -y && \
    apt-get install -y python3 python3-pip git
COPY . /
RUN pip install --no-cache-dir -r requirements.txt
RUN pip install gunicorn
ENV GUNICORN_CMD_ARGS="--workers=1 --bind=0.0.0.0:8201"
EXPOSE 8212
ENV FLASK_ENV=production
CMD [ "gunicorn", "main:app" ]
```
- [X]   docker-compose.yml
```yml
version: '3'
services:
        web:
                image: clubhub
                build: .
                env_file:
                        - .env # This file is optional; defaults will be used if it does not exist
                ports:
                        - "8201:8201"
                volumes:
                        - ./instance:/instance
                restart: unless-stopped
```
- [X]   nginx_file : preparing this file for reverse proxy (essentially sending info form internet, to application, and back to requestor)
```python
server {
    listen 80;
    listen [::]:80;
    server_name clubhub.stu.nighthawkcodingsociety.com ; # Change server name to the one on R53
    # Configure CORS Headers
    location / { 
        proxy_pass http://localhost:8201; # Change port to port on docker
        # Simple requests
        if ($request_method ~* "(GET|POST|PUT|DELETE)") { # Customize Request methods based on your needs
                add_header "Access-Control-Allow-Origin"  *;
        }
        # Preflighted requests 
        if ($request_method = OPTIONS ) {
                add_header "Access-Control-Allow-Origin"  "https://github.com/magic005/Club_Hub_Frontend" always;
                add_header "Access-Control-Allow-Methods" "GET, POST, PUT, DELETE, OPTIONS, HEAD"; # Make sure the request methods above match here
                add_header "Access-Control-Allow-Headers" "Authorization, Origin, X-Requested-With, Content-Type, Accept";
                return 200;
        }
    }
}
```
- [X]   Frontend settings: prepare the frontend to access our domain and ports to match our localhost, port 8201, and domain settings. this example is modifications made in config.js file on frontend
```python
export var pythonURI;
if (location.hostname === "localhost") {
        pythonURI = "http://localhost:8201";
} else if (location.hostname === "127.0.0.1") {
        pythonURI = "http://127.0.0.1:8201";
} else {
        pythonURI =  "clubhub.stu.nighthawkcodingsociety.com";
}
```
Setting Up AWS EC2 Instance

- [X]   Log in to AWS and navigate to EC2
- [X]   Launch an appropriate EC2 instance (CSP)
- [X]   Connect to the instance via SS

Preparing Docker on Localhost

- [X]   Open the backend project in VSCode
- [X]   Ensure Docker files are correctly configured
- [X]   Run Docker locally and verify the service is accessible
- [X]   Push the latest changes to GitHub

**nginx Setup**
1. ***Navigate to nginx***: cd /etc/nginx/sites-available<br>
2. ***Create an nginx config file***: sudo nano neptune_backend<br>
3. ***Activate configuration***: cd /etc/nginx/sites-enabled, then sudo ln -s /etc/nginx/sites-available/neptune_backend /etc/nginx/sites-enabled<br>
4. ***Validate***: sudo nginx -t<br>
5. ***Restart nginx***: sudo systemctl restart nginx<br>

Deploying to AWS EC2

- Clone backend repo: git clone https://github.com/luojonah/clubhub_backend.git
- Navigate to repo: cd clubhub_backend
- Build site: docker-compose up -d --build
- Test site: curl localhost:8201

Run command below and follow prompts:

sudo certbot --nginx


Updating and Redeploying Code
On Local Machine:

1. Run git pull before making changes
2. Open terminal in VSCode and run python main.py
3. Make changes that are needed
4. Commit the changes locally
5. Test docker-compose up or sudo docker-compose up in VSCode terminal
6. Sync change from UI/git push from terminal

Pulling Changes into AWS EC2 deployment:

1. Navigate to repo: cd ~/neptune_backend
2. docker-compose down
3. git pull
4. Rebuild docker container: docker-compose up -d --build

Troubleshooting Deployment Issues

1. Check if the application is running correctly
2. Verify running Docker containers
3. Inspect logs for errors
4. Confirm proper port forwarding and domain configuration

![Image](https://github.com/user-attachments/assets/f697feb5-3aec-47eb-9854-20a1c3c4ce58)